In [1]:
import os 
import subprocess as sp
# --------------------------------------------------
def season_dictionary():
    
    season_dict = {
        '10': {
            'scanner3DTop': {
                'containers': {
                    'gantry_notifications': {
                        'simg': 'gantry_notifications.simg', 
                        'dockerhub_path': 'docker://phytooracle/slack_notifications:latest'
                    },
                    'preprocessing': {
                        'simg': '3d_preprocessing.simg', 
                        'dockerhub_path': 'docker://phytooracle/3d_preprocessing:latest'
                    },
                    'sequential_alignment': {
                        'simg': '3d_sequential_align.simg',
                        'dockerhub_path': 'docker://phytooracle/3d_sequential_align:latest'
                    }
                },
                'workflow_1': {
                    'commands': [
                    'jx2json main_workflow_phase1.jx -a bundle_list.json > main_workflow_phase1.json', 
                    'makeflow -T wq --json main_workflow_phase1.json -a -N phytooracle_3d -M phytooracle_3d -r 3 -p 0 -dall -o dall.log $@'
                    ],
                    'outputs': {
                        'pipeline_out': 'preprocessing_out',
                        'tag': 'preprocessed',
                        'outdir': 'preprocessing'
                    }
                },
                'intermediate': {
                    'commands': {
                    'singularity run 3d_sequential_align.simg -i preprocessing_out/ -o sequential_alignment_out/'
                    },
                    'outputs': {
                        'pipeline_out': 'sequential_alignment_out',
                        'tag': 'aligned',
                        'outdir': 'alignment'
                    }
                },
                'workflow_2': {
                    'commands': {
                        'jx2json main_workflow_phase-2.jx -a bundle_list.json > main_workflow_phase2.json', 
                        'makeflow -T wq --json main_workflow_phase2.json -a -r 2 -M phytooracle_3d -N phytooracle_3d -p 60221 -dall -o dall.log --disable-cache $@'
                    },
                    'outputs': {
                        'pipeline_out': '',
                        'tag': '',
                        'outdir': ''
                    }
                }
            }
        }
    }

    return season_dict

In [2]:
season_dict = season_dictionary()

In [4]:
for item in season_dict['10']['scanner3DTop']['workflow_1']['commands']:
    print(item)

jx2json main_workflow_phase1.jx -a bundle_list.json > main_workflow_phase1.json
makeflow -T wq --json main_workflow_phase1.json -a -N phytooracle_3d -M phytooracle_3d -r 3 -p 0 -dall -o dall.log $@


In [ ]:
# --------------------------------------------------
def run_workflow_1(season, sensor, season_dict):

    home = os.path.join(os.path.expanduser('~'), 'cctools-7.1.12-x86_64-centos7', 'bin/')
    sp.run(["sbatch", "worker_scripts/po_work_puma_slurm.sh"])

    for item in season_dict[season][sensor]['workflow_1']['commands']:
        cmd = ''.join([home, item])
        print(cmd)